# 🧩 Noisy Particle Segmentation with Autoencoder
This notebook demonstrates:
- Generating the noisy dataset
- Training the convolutional autoencoder
- Visualizing reconstructions
- Exploring the latent space with PCA

In [ ]:
# Install dependencies (if needed)
!pip install torch torchvision matplotlib scikit-image scikit-learn

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from src.dataset import NoisySegDataset
from src.model import Autoencoder1D
from src.utils import weighted_mse
from src.visualize import visualize_reconstructions, visualize_latent_space

## 1. Create dataset

In [ ]:
dataset = NoisySegDataset(n_samples=500, img_size=128)
print("Dataset size:", len(dataset))

# Show a sample
x, y = dataset[0]
plt.imshow(x.squeeze(), cmap='gray')
plt.title("Example Noisy Input")
plt.axis('off')
plt.show()

## 2. Train Autoencoder

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Autoencoder1D(latent_dim=512).to(device)
loader = DataLoader(dataset, batch_size=32, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 20  # keep small for demo
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = weighted_mse(out, y, weight=20.0)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} Loss: {total_loss/len(loader):.4f}")

## 3. Visualize Reconstructions

In [ ]:
visualize_reconstructions(model, dataset, device, n=3)

## 4. Latent Space Visualization

In [ ]:
visualize_latent_space(model, dataset, device)

🎉 Done! You can also sample new latent vectors manually using PCA directions for generative exploration.